In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Define the seasons I need data for
seasons = [2024]

In [200]:
from db_builder import update_results_and_club_values


results_df, club_values_df = update_results_and_club_values(seasons, True)
results_df

Updated 1140 rows in the football_data_season_results table


,season,home,away,utc_date,status,matchday,home_score,away_score,home_outcome,away_outcome
0,2022,Liverpool FC,Chelsea FC,2023-01-21 12:30:00+00:00,FINISHED,21,0.0,0.0,1.0,1.0
1,2022,West Ham United FC,Newcastle United FC,2023-04-05 19:00:00+00:00,FINISHED,7,1.0,5.0,0.0,3.0
2,2022,Manchester City FC,Chelsea FC,2023-05-21 15:00:00+00:00,FINISHED,37,1.0,0.0,3.0,0.0
3,2022,Crystal Palace FC,Nottingham Forest FC,2023-05-28 15:30:00+00:00,FINISHED,38,1.0,1.0,1.0,1.0
4,2022,Everton FC,AFC Bournemouth,2023-05-28 15:30:00+00:00,FINISHED,38,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1135,2024,Brighton & Hove Albion FC,Southampton FC,2024-11-30 00:00:00+00:00,SCHEDULED,13,NaN,NaN,NaN,NaN
1136,2024,Brentford FC,Leicester City FC,2024-11-30 00:00:00+00:00,SCHEDULED,13,NaN,NaN,NaN,NaN
1137,2024,Southampton FC,Liverpool FC,2024-11-23 00:00:00+00:00,SCHEDULED,12,NaN,NaN,NaN,NaN
1138,2024,West Ham United FC,Arsenal FC,2024-11-30 00:00:00+00:00,SCHEDULED,13,NaN,NaN,NaN,NaN


In [201]:
club_values_df

Updated 72 rows in the transfermarkt_club_values table


,club,season,value
0,Manchester City FC,2022,1.145950e+09
1,Crystal Palace FC,2022,3.230500e+08
2,Burnley FC,2022,1.933500e+08
3,Wolverhampton Wanderers FC,2022,4.976500e+08
4,Sheffield United FC,2022,1.185500e+08
...,...,...,...
67,Manchester City FC,2024,1.408200e+09
68,West Ham United FC,2024,4.545000e+08
69,Liverpool FC,2024,9.493000e+08
70,Newcastle United FC,2024,6.566500e+08


In [202]:
from db_builder import update_manager_tenure


managers_df = update_manager_tenure()
managers_df

Updated 477 rows in the premier_league_managers table


,manager,club,start,end,duration_days,years,incumbent,caretaker,incumbent_not_in_league
0,George Graham,Arsenal,1986-05-14,1995-02-21,3205,1992–1995,False,False,False
1,Stewart Houston,Arsenal,1995-02-22,1995-06-08,106,1995,False,True,False
2,Bruce Rioch,Arsenal,1995-06-08,1996-08-12,431,1995–1996,False,False,False
3,Stewart Houston,Arsenal,1996-08-12,1996-09-13,32,1996,False,True,False
4,Pat Rice,Arsenal,1996-09-13,1996-09-30,17,1996,False,True,False
...,...,...,...,...,...,...,...,...,...
472,Nuno Espírito Santo,Wolverhampton Wanderers,2017-05-31,2021-05-23,1453,2018–2021,False,False,False
473,Bruno Lage,Wolverhampton Wanderers,2021-06-09,2022-10-02,1149,2021–2022,False,False,False
474,Steve Davis,Wolverhampton Wanderers,2022-10-03,2022-11-13,41,2022,False,True,False
475,Julen Lopetegui,Wolverhampton Wanderers,2022-11-14,2023-08-08,267,2022–2023,False,False,False
